# Analyse et visualisation de données avec Python
## Combiner des DataFrame avec Pandas
Questions
* Peut-on travaillers avec plusieurs sources de données?
* Comment combiner les données de deux DataFrames?

Objectifs
* Combiner les données de plusieurs fichiers en utilisant `concat` et `merge`.
* Combiner deux DataFrames utilisant un identifiant commun.

## Charger nos données

In [ ]:
# Charger le module pandas
import pandas as pd

# Charger les données
surveys_df = pd.read_csv("../data/surveys.csv")

## Concaténer des DataFrames

In [ ]:
# Sélectionner les 10 premiers enregistrements
premiers10 = surveys_df.head(10)
premiers10

In [ ]:
# Sélectionner les 10 derniers enregistrements
derniers10 = surveys_df.tail(10)
derniers10

In [ ]:
# Concaténer les dataframes verticalement
vertical = pd.concat([premiers10, derniers10], axis=0)
vertical

In [ ]:
# Réinitaliser l'index du dataframe
# L'option drop=True évite l'ajout d'une colonne avec l'ancien index
vertical = vertical.reset_index(drop=True)
vertical

### Écrire le résultat dans un fichier CSV

In [ ]:
# Omettre l'index
vertical.to_csv('surveys_debut_fin.csv', index=False)

In [ ]:
# Charger le nouveau fichier CSV
debut_fin = pd.read_csv('surveys_debut_fin.csv')
debut_fin

## Exercice - Concaténer des DataFrames
Dans `surveys_df`, sélectionnez individuellement les enregistrements des années 2001 et 2002. Concaténez les deux dataframes verticalement. Sauvegardez le tableau des moyennes dans un CSV et le recharger.

In [ ]:
# Obtenir les données pour chaque année
annee2001 = surveys_df[surveys_df['year'] == 2001]
annee2002 = surveys_df[surveys_df['year'] == 2002]

# Concaténer verticalement
deux_annees = pd.concat([annee2001, annee2002], axis=0)

In [ ]:
# Calculer la moyenne par année et par sexe
poids_annee = deux_annees.groupby(['year', 'sex'])['weight'].mean().unstack()
poids_annee

In [ ]:
# Écrire dans un fichier - garder l'index "year" cette fois-ci
poids_annee.to_csv("poids_par_annee.csv", index=True)

# Relire les données, fournir le nom de l'index
pd.read_csv("poids_par_annee.csv", index_col='year')

## Joindre deux DataFrames

In [ ]:
# Importer un sous-ensemble des espèces pour cet exemple
trois_especes = pd.read_csv('../data/speciesSubset.csv')
trois_especes

### Identifier les clés de jonction

In [ ]:
premiers10.columns

In [ ]:
trois_especes.columns

### Une intersection ou "inner join"

![Inner join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/inner-join.png)

In [ ]:
intersection = pd.merge(left=premiers10, right=trois_especes,
                        left_on='species_id', right_on='species_id')
# Quelle est la taille de la jonction?
intersection.shape

In [ ]:
intersection

### Jonction de gauche

![Left join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/left-join.png)

In [ ]:
jonc_gauche = pd.merge(left=premiers10, right=trois_especes,
                       how='left', on='species_id')
# Quelle est la taille de la jonction?
jonc_gauche.shape

In [ ]:
jonc_gauche

### Les autres types de jonction
* `how='right'` : toutes les rangées du second DataFrame sont gardées
* `how='outer'` : équivalent d'une union, toutes les rangées sont gardées

## Exercice - Joindre toutes les données
`1`. Créez un nouveau DataFrame tel que tous les enregistrements de `surveys.csv` seront gardés, mais certains enregistrement seront joints aux informations correspondantes de `species.csv` (soit le fichier complet à charger).

In [ ]:
species_df = pd.read_csv("../data/species.csv")
jonc_gauche = pd.merge(left=surveys_df, right=species_df, how='left', on='species_id')
jonc_gauche.shape

`2`. Calculez et créez un graphique (*bar-plot*) montrant le nombre d'enregistrements (soit le nombre de `record_id`) par type de `taxa` pour chaque site (`plot_id`).

In [ ]:
taxa_site = jonc_gauche.groupby(['plot_id', 'taxa'])['record_id'].count().unstack()
taxa_site.head()

In [ ]:
taxa_site.plot(kind="bar", stacked=True)

`3`. Calculez et créez un graphique (bar-plot) montrant le nombre de différents `taxa` par type de sexe pour chaque site (`plot_id`).

In [ ]:
# Nettoyage des données
jonc_gauche.loc[jonc_gauche['sex'].isnull(), 'sex'] = "F|M"
jonc_gauche.loc[~jonc_gauche['sex'].isin(["F", "F|M", "M"]), 'sex'] = "F|M"

In [ ]:
ntaxa_sex_site = jonc_gauche.groupby(['plot_id',
                                      'sex'])['taxa'].nunique().reset_index(level=1)
ntaxa_sex_site.head()

In [ ]:
# Utiliser pivot_table() au lieu de unstack()
ntaxa_sex_site = ntaxa_sex_site.pivot_table(values='taxa', columns='sex',
                                            index=ntaxa_sex_site.index)
ntaxa_sex_site.head()

In [ ]:
ntaxa_sex_site.plot(kind="bar", stacked=True, legend=False)